In [ ]:
import pickle

with open("AndroidModules.pkl", "rb") as target:
    third_party_calls, third_party_total_call, detected_libs_infos = pickle.load(target)

In [ ]:
# Version 1 
apis_requests_location_permission = {}
API_loc_freq = {}
all_methods = 0
all_classes = 0
all_apis = 0
for call in detected_libs_infos:
    permissions = call[3]
    location_perms = [p for p in permissions if "location" in p.lower()]
    if len(location_perms) == 0:
        continue
        
    # API's according to their number of requesting location permission.
    if call[0] not in API_loc_freq:
        API_loc_freq[call[0]] = 0
    API_loc_freq[call[0]] += 1   
    
    if call[0] not in apis_requests_location_permission:
        apis_requests_location_permission[call[0]] = {}
        all_apis += 1
    if call[1] not in apis_requests_location_permission[call[0]]:
        apis_requests_location_permission[call[0]][call[1]] = {}
        all_classes += 1
    if call[2] not in apis_requests_location_permission[call[0]][call[1]]:
        apis_requests_location_permission[call[0]][call[1]][call[2]] = 0
        all_methods += 1
    apis_requests_location_permission[call[0]][call[1]][call[2]] += 1

In [ ]:
# Version 2: Elminate obfuscated classes.
apis_requests_location_permission = {}
API_loc_freq = {}
for call in detected_libs_infos:
    permissions = call[3]
    location_perms = [p for p in permissions if "location" in p.lower()]
    if len(location_perms) == 0:
        continue
        
    class_name = call[1].split("/")[-1].replace(";","")
    if class_name.lower() == class_name:
        continue
        
    # API's according to their number of requesting location permission.
    if call[0] not in API_loc_freq:
        API_loc_freq[call[0]] = 0
    API_loc_freq[call[0]] += 1   
    
    
    if call[0] not in apis_requests_location_permission:
        apis_requests_location_permission[call[0]] = {}
    if call[1] not in apis_requests_location_permission[call[0]]:
        apis_requests_location_permission[call[0]][call[1]] = {}
    if call[2] not in apis_requests_location_permission[call[0]][call[1]]:
        apis_requests_location_permission[call[0]][call[1]][call[2]] = 0
    apis_requests_location_permission[call[0]][call[1]][call[2]] += 1

In [ ]:
# Version 3: Elminate obfuscated classes and methods.
apis_requests_location_permission = {}
API_loc_freq = {}
for call in detected_libs_infos:
    permissions = call[3]
    location_perms = [p for p in permissions if "location" in p.lower()]
    if len(location_perms) == 0:
        continue
    
    # Eliminate obfuscated class 
    class_name = call[1].split("/")[-1].replace(";","")
    if class_name.lower() == class_name:
        continue
    
    # Eliminate obfuscated method 
    class_method = call[2].split()[-1].split("(")[0].strip()
    if len(class_method) < 2:
        continue
    #print(class_method, class_method.split()[-1].split("(")[0].strip())

    # API's according to their number of requesting location permission.
    if call[0] not in API_loc_freq:
        API_loc_freq[call[0]] = 0
    API_loc_freq[call[0]] += 1   
    
    
    if call[0] not in apis_requests_location_permission:
        apis_requests_location_permission[call[0]] = {}
    if call[1] not in apis_requests_location_permission[call[0]]:
        apis_requests_location_permission[call[0]][call[1]] = {}
    if call[2] not in apis_requests_location_permission[call[0]][call[1]]:
        apis_requests_location_permission[call[0]][call[1]][call[2]] = 0
    apis_requests_location_permission[call[0]][call[1]][call[2]] += 1

In [ ]:
#pprint.pprint(apis_requests_location_permission["Lcom/mopub"])
#pprint.pprint(apis_requests_location_permission["Lcom/inmobi"])
#pprint.pprint(apis_requests_location_permission["Lcom/unity3d"])

In [ ]:
# For printing purposes
import pprint 
sorted_APIS = {k: v for k, v in sorted(API_loc_freq.items(), key=lambda item: item[1], reverse=True)}

non_obsfucated_methods = 0
non_obsfucated_classes = 0
non_obsfucated_apis = 0
for k in sorted_APIS:
    non_obsfucated_apis += 1
    print(k, ",,,", sorted_APIS[k])
    for c in apis_requests_location_permission[k]:
        non_obsfucated_classes += 1
        print(",", c, ",,")
        for m in apis_requests_location_permission[k][c]:
            non_obsfucated_methods += 1
            print(","*2, m, ",", apis_requests_location_permission[k][c][m])
        #print("\t", "*"*10)
    #print("-"*80)
print(non_obsfucated_apis, non_obsfucated_classes, non_obsfucated_methods)
print(all_apis, all_classes, all_methods)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def capture_artifacts(lib, i=1, limit=10):
    url = "https://mvnrepository.com/search?q={}&p={}".format(lib, i)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    response = requests.get(url, headers=headers)
    print("response", response.status_code)
    soup = BeautifulSoup(response.content, "html.parser")
    artifact_divs = soup.find_all("div", {"class": "im"})
    
    if len(soup.find_all("div", {"class": "im"})) == 0:
        return []
    if i == limit:
        return []
    
    artifacts = []
    for div in artifact_divs:
        path = div.find("h2", {"class": "im-title"}).find("a")["href"]
        artifacts.append((path.split("/")[-2], path.split("/")[-1]))
    time.sleep(10)
    return artifacts + capture_artifacts(lib, i+1, limit)

c = 0
api_artifacts_coarse = {}
for k in sorted_APIS:
    group = ".".join(k.split("/")[1:])
    artifacts = capture_artifacts(group)
    api_artifacts[k] = artifacts
    pprint.pprint(group)
    pprint.pprint(artifacts)
    print("-"*80)


In [11]:
import requests
from bs4 import BeautifulSoup
import time

def capture_artifacts(lib, group, i=1, limit=10):
    url = "https://mvnrepository.com/search?q={}&d={}&p={}".format(lib, group,  i)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    response = requests.get(url, headers=headers)
    print("response", response.status_code)
    soup = BeautifulSoup(response.content, "html.parser")
    artifact_divs = soup.find_all("div", {"class": "im"})
    
    if len(soup.find_all("div", {"class": "im"})) == 0:
        return []
    if i == limit:
        return []
    
    artifacts = []
    for div in artifact_divs:
        path = div.find("h2", {"class": "im-title"}).find("a")["href"]
        artifacts.append((path.split("/")[-2], path.split("/")[-1]))
    time.sleep(10)
    return artifacts + capture_artifacts(lib, group,  i+1, limit)

c = 0
api_artifacts_fine = {}
for k in sorted_APIS:
    group = k[1:].replace("/", ".")
    lib_key = ".".join(k.split("/")[1:])
    artifacts = capture_artifacts(lib_key, group)
    api_artifacts[k] = artifacts
    pprint.pprint(group)
    pprint.pprint(artifacts)
    print("-"*80)

response 200
'com.amplitude'
[('com.amplitude', 'android-sdk')]
--------------------------------------------------------------------------------
response 200
'org.cocos2dx'
[]
--------------------------------------------------------------------------------
response 200
'com.MASTAdView'
[]
--------------------------------------------------------------------------------
response 200
'com.doodlemobile.gamecenter'
[]
--------------------------------------------------------------------------------


In [16]:
import pandas as pd
df = pd.DataFrame(api_artifacts_fine)
df.head()

ValueError: arrays must all be same length

In [32]:
for api in sorted_APIS:
    if api in api_artifacts_fine:
        print(api,",,")
        for g,a in api_artifacts_fine[api]:
            print(",",g,",",a)

NOT_FOUND ,,
Lcom/inmobi ,,
, com.inmobi.monetization , inmobi-ads
, com.inmobi.monetization , inmobi-mediation
, com.inmobi.hive.test , hiveunit-mr2
, com.inmobi.monetization , inmobi-mopub-plugin
, com.inmobi.monetization , inmobi-AB-plugin-mopub
, com.inmobi.monetization , inmobi-AB-plugin-dfp
, com.inmobi.monetization , inmobi-core
Lcom/unity3d ,,
, com.unity3d.ads , unity-ads
Lcom/mopub ,,
, com.mopub , mopub-sdk
, com.mopub.volley , mopub-volley
, com.mopub , mopub-sdk-banner
, com.mopub , mopub-sdk-interstitial
, com.mopub , mopub-sdk-base
, com.mopub , mopub-sdk-rewardedvideo
, com.mopub , mopub-sdk-native-static
, com.mopub , mopub-sdk-native-video
, com.mopub , libAvid-mopub
, com.mopub , mopub-sdk-fullscreen
, com.mopub.mediation , facebookaudiencenetwork
, com.mopub.mediation , vungle
, com.mopub , omsdk-android
, com.mopub.mediation , chartboost
, com.mopub.mediation , unityads
, com.mopub.mediation , pangle
, com.mopub.mediation , onebyaol
, com.mopub.mediation , verizon


In [18]:
import pickle
with open("api_artifacts_mvnrepo.pkl", "wb") as f:
    pickle.dump(api_artifacts_fine, f)  

In [28]:
# For printing purposes
import pprint 
sorted_APIS = {k: v for k, v in sorted(API_loc_freq.items(), key=lambda item: item[1], reverse=True)}

non_obsfucated_methods = 0
non_obsfucated_classes = 0
non_obsfucated_apis = 0
for k in sorted_APIS:
    non_obsfucated_apis += 1
    if k in api_artifacts_fine:
        print(k, ",,,", sorted_APIS[k], ",", "---".join(["{}::{}".format(g,a) for g,a in api_artifacts_fine[k]]))
    else:
        print(k, ",,,", sorted_APIS[k], ",")
    for c in apis_requests_location_permission[k]:
        non_obsfucated_classes += 1
        print(",", c, ",,,")
        for m in apis_requests_location_permission[k][c]:
            non_obsfucated_methods += 1
            print(","*2, m, ",", apis_requests_location_permission[k][c][m], ",")
            
        #print("\t", "*"*10)
    #print("-"*80)
print(non_obsfucated_apis, non_obsfucated_classes, non_obsfucated_methods)
print(all_apis, all_classes, all_methods)

NOT_FOUND ,,, 92 , 
, .class Lcom/vnaa/etnh157744/UserDetails; ,,,
,, .method getLocation()Landroid/location/Location; , 5 ,
, .class public Lcn/dm/networktool/tools/UserDeviceInfo; ,,,
,, .method protected static getLocationAreaCode(Landroid/content/Context;)Ljava/lang/String; , 1 ,
,, .method protected static getCellID(Landroid/content/Context;)Ljava/lang/String; , 1 ,
, .class Lcom/doodlemobile/gamecenter/DLocation; ,,,
,, .method public stopLocation()V , 12 ,
, .class public Lcom/tune/location/TuneLocationListener; ,,,
,, .method public isProviderEnabled()Z , 3 ,
, .class public Lcom/fusepowered/location/LocationService; ,,,
,, .method public getRawLastKnownLocation(Landroid/content/Context;)Landroid/location/Location; , 1 ,
, .class public Lcom/amplitude/api/DeviceInfo; ,,,
,, .method public getMostRecentLocation()Landroid/location/Location; , 1 ,
, .class public Lcom/amplitude/api/Amplitude; ,,,
,, .method private static getMostRecentLocation()Landroid/location/Location; , 1 ,
, 